In [50]:
import pandas as pd

df = pd.read_csv("../assets/elaws_links.csv")
df

,title,link_to_page,link_to_doc_file
0,"Food Safety and Quality Act, 2001, S.O. 2001, ...",https://www.ontario.ca/laws/statute/01f20,https://www.ontario.ca/laws/docs/01f20_e.doc
1,"Nutrient Management Act, 2002, S.O. 2002, c. 4",https://www.ontario.ca/laws/statute/02n04,https://www.ontario.ca/laws/docs/02n04_e.doc
2,"Animal Health Act, 2009, S.O. 2009, c. 31",https://www.ontario.ca/laws/statute/09a31,https://www.ontario.ca/laws/docs/09a31_e.doc
3,"Liquor Licence and Control Act, 2019, S.O. 201...",https://www.ontario.ca/laws/statute/19l15b,https://www.ontario.ca/laws/docs/19l15b_e.doc
4,"Commodity Board Members Act, R.S.O. 1990, c. C.18",https://www.ontario.ca/laws/statute/90c18,https://www.ontario.ca/laws/docs/90c18_e.doc
5,"Education Act, R.S.O. 1990, c. E.2",https://www.ontario.ca/laws/statute/90e02,https://www.ontario.ca/laws/docs/90e02_e.doc
6,"Farm Products Payments Act, R.S.O. 1990, c. F.10",https://www.ontario.ca/laws/statute/90f10,https://www.ontario.ca/laws/docs/90f10_e.doc
7,"Health Protection and Promotion Act, R.S.O. 19...",https://www.ontario.ca/laws/statute/90h07,https://www.ontario.ca/laws/docs/90h07_e.doc
8,"Highway Traffic Act, R.S.O. 1990, c. H.8",https://www.ontario.ca/laws/statute/90h08,https://www.ontario.ca/laws/docs/90h08_e.doc
9,"Milk Act, R.S.O. 1990, c. M.12",https://www.ontario.ca/laws/statute/90m12,https://www.ontario.ca/laws/docs/90m12_e.doc


In [54]:
from docling.document_converter import DocumentConverter, PdfFormatOption

from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.datamodel.base_models import InputFormat

# force no OCR
pipeline_options = PdfPipelineOptions()
pipeline_options.do_ocr = False

converter = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
    }
)

result = converter.convert('../assets/elaws_pdfs/01f20_e.pdf')
print(result.document.export_to_markdown())

IsADirectoryError: [Errno 21] Is a directory: '../assets/elaws_pdfs'

In [67]:
from docling.chunking import HybridChunker

chunker = HybridChunker()
chunk_iter = chunker.chunk(result.document)

Token indices sequence length is longer than the specified maximum sequence length for this model (523 > 512). Running this sequence through the model will result in indexing errors


In [68]:
chunks = list(chunk_iter)
chunks[15].model_dump()

{'text': '(2)  A director shall not refuse to issue a licence to an applicant unless,\n(a) before refusing to issue the licence, the director serves a written notice on the applicant stating that the applicant may request a hearing by the director within the prescribed time; and\n(b)  the director has held the hearing if the applicant requests one within the prescribed time.  2001, c. 20, s. 5 (2).',
 'meta': {'schema_name': 'docling_core.transforms.chunker.DocMeta',
  'version': '1.0.0',
  'doc_items': [{'self_ref': '#/texts/137',
    'parent': {'cref': '#/groups/16'},
    'children': [],
    'content_layer': <ContentLayer.BODY: 'body'>,
    'label': <DocItemLabel.LIST_ITEM: 'list_item'>,
    'prov': [{'page_no': 4,
      'bbox': {'l': 60.025001525878906,
       't': 205.72999572753906,
       'r': 345.6499938964844,
       'b': 197.1199951171875,
       'coord_origin': <CoordOrigin.BOTTOMLEFT: 'BOTTOMLEFT'>},
      'charspan': (0, 75)}]},
   {'self_ref': '#/texts/138',
    'parent': 